# Chapter 27: Domain‑Driven Design (DDD) Concepts

As applications grow in complexity, especially those with rich business logic, a well‑designed domain model becomes critical. **Domain‑Driven Design (DDD)** is an approach to software development that focuses on modeling the core business domain and its logic, and it provides a set of patterns and practices for building complex systems.

DDD is not a framework or a library – it's a mindset and a collection of strategic and tactical patterns. In this chapter, we'll focus on the **tactical patterns** that you can apply directly in your C# code:

- **Entities** – objects with an identity and a lifecycle.
- **Value Objects** – immutable objects defined by their attributes.
- **Aggregates** – clusters of entities and value objects treated as a unit.
- **Repositories** – abstractions for retrieving and persisting aggregates.
- **Domain Events** – capturing something meaningful that happened in the domain.

We'll also touch on the **Ubiquitous Language** and how to keep your code aligned with business concepts.

By the end, you'll be able to model complex business domains in a way that is both expressive and maintainable.

---

## 27.1 What is Domain‑Driven Design?

DDD was introduced by Eric Evans in his book *Domain‑Driven Design: Tackling Complexity in the Heart of Software*. The core idea is that **the heart of software is the domain** – the area of business activity your application supports. To build successful software, you need to deeply understand that domain and reflect its nuances in your code.

### The Ubiquitous Language

A key practice in DDD is creating a **ubiquitous language** – a common, rigorous language shared by developers and domain experts. This language is used in conversations, documentation, and the code itself. When a term appears in the code, it has the same meaning as in the business discussions.

For example, if the business talks about an "Order" being "submitted", your code should have an `Order` class and a `Submit` method – not `PlaceOrder` or `Create` if those don't match the business language. This alignment reduces misunderstandings and makes the code self‑documenting.

### Strategic vs. Tactical Patterns

DDD provides both strategic patterns (like Bounded Contexts, Context Maps) and tactical patterns (like Entities, Value Objects). Strategic patterns help you decompose a large system into manageable parts. Tactical patterns help you build rich models within each bounded context. In this chapter, we focus on tactical patterns.

---

## 27.2 Entities

An **Entity** is an object that has a distinct identity that runs through time and often across different representations. Two entities are considered equal if their identities match, even if their other attributes differ.

### Characteristics

- Has an identifier (e.g., `Id` property).
- Mutable – its state can change over time.
- Equality based on identity, not on attribute values.

### Example: Order Entity

```csharp
public class Order
{
    public OrderId Id { get; private set; }
    public string CustomerName { get; private set; }
    public DateTime OrderDate { get; private set; }
    private readonly List<OrderItem> _items = new List<OrderItem>();
    public IReadOnlyList<OrderItem> Items => _items.AsReadOnly();
    public OrderStatus Status { get; private set; }

    private Order() { } // for ORM

    public Order(string customerName)
    {
        Id = new OrderId(Guid.NewGuid());
        CustomerName = customerName;
        OrderDate = DateTime.UtcNow;
        Status = OrderStatus.Pending;
    }

    public void AddItem(Product product, int quantity, decimal price)
    {
        // Business rules: can only add items while pending
        if (Status != OrderStatus.Pending)
            throw new InvalidOperationException("Cannot add items to a non-pending order");

        var item = new OrderItem(product, quantity, price);
        _items.Add(item);
    }

    public void Submit()
    {
        if (Status != OrderStatus.Pending)
            throw new InvalidOperationException("Order already submitted");
        if (!_items.Any())
            throw new InvalidOperationException("Cannot submit empty order");

        Status = OrderStatus.Submitted;
        // Raise domain event
    }
}

public enum OrderStatus
{
    Pending,
    Submitted,
    Shipped,
    Cancelled
}

// Identity as a value object (to make it strongly typed)
public record OrderId(Guid Value);
```

Notice:
- `OrderId` is a value object that wraps a `Guid`. This makes the identity explicit and type‑safe.
- The entity has behavior (methods) that enforce business rules.
- The `Id` is immutable after creation.
- The constructor ensures a valid initial state.

---

## 27.3 Value Objects

A **Value Object** is an immutable object whose equality is based on the values of its properties, not on an identity. Examples: `Money`, `Address`, `DateRange`.

### Characteristics

- Immutable – once created, it cannot change.
- No identity – two value objects with the same properties are considered equal.
- Can be easily replaced with a new instance if changes are needed.
- Often used to model concepts that don't have a lifecycle.

### Example: Address and Money

```csharp
public record Address
{
    public string Street { get; }
    public string City { get; }
    public string ZipCode { get; }
    public string Country { get; }

    public Address(string street, string city, string zipCode, string country)
    {
        Street = street;
        City = city;
        ZipCode = zipCode;
        Country = country;
    }
}

public record Money
{
    public decimal Amount { get; }
    public string Currency { get; }

    public Money(decimal amount, string currency)
    {
        if (amount < 0) throw new ArgumentException("Amount cannot be negative");
        Amount = amount;
        Currency = currency ?? throw new ArgumentNullException(nameof(currency));
    }

    public Money Add(Money other)
    {
        if (other.Currency != Currency)
            throw new InvalidOperationException("Cannot add different currencies");
        return new Money(Amount + other.Amount, Currency);
    }

    // Other operations: Subtract, Multiply, etc.
}
```

Using `record` (C# 9+) is perfect for value objects – they provide value‑based equality, immutability, and a concise syntax.

### Why Use Value Objects?

- **Clarity** – instead of passing around primitives (e.g., `decimal amount`), you pass a `Money` object, which encapsulates currency and validation.
- **Maintainability** – if you later need to add validation or formatting logic, it lives in one place.
- **Type safety** – prevents mixing different currencies or passing a negative amount where not allowed.

---

## 27.4 Aggregates

An **Aggregate** is a cluster of associated objects (entities and value objects) that are treated as a unit for data changes. Each aggregate has a root – the **Aggregate Root** – which is the only object that external objects are allowed to hold references to.

### Why Aggregates?

- They enforce consistency boundaries. All invariants (business rules) for the objects inside the aggregate are enforced by the root.
- They simplify persistence – you load and save the entire aggregate as a whole.

### Rules of Thumb

- The aggregate root is an entity with global identity.
- Other entities inside the aggregate have local identities (only unique within the aggregate).
- External objects can only reference the root, not internal entities.
- Changes to the aggregate are made through the root, which ensures invariants.

### Example: Order as an Aggregate Root

In our order example, `Order` is the aggregate root. It contains `OrderItem` entities. No external object should hold a reference to an `OrderItem` directly; all operations go through the `Order`.

```csharp
public class OrderItem
{
    public Product Product { get; }
    public int Quantity { get; private set; }
    public Money Price { get; }

    internal OrderItem(Product product, int quantity, Money price)
    {
        Product = product;
        Quantity = quantity;
        Price = price;
    }

    public void UpdateQuantity(int newQuantity)
    {
        if (newQuantity <= 0) throw new ArgumentException("Quantity must be positive");
        Quantity = newQuantity;
    }
}
```

Note that `OrderItem`'s constructor is `internal` – only the aggregate root (within the same assembly) can create it.

---

## 27.5 Repositories

A **Repository** mediates between the domain and data mapping layers, acting like an in‑memory collection of aggregate roots. It provides a way to retrieve and persist aggregates without exposing persistence details to the domain.

### Characteristics

- Repository interfaces are defined in the domain layer, but implementations are in the infrastructure layer (dependency inversion).
- They work only with aggregate roots.
- They often provide methods like `GetById`, `Add`, `Remove`, and sometimes query methods.

### Example: OrderRepository

```csharp
public interface IOrderRepository
{
    Order Get(OrderId id);
    void Add(Order order);
    void Update(Order order);
    void Remove(Order order);
    IEnumerable<Order> GetByCustomer(string customerName);
}
```

The implementation (e.g., using Entity Framework Core) resides in the infrastructure project.

```csharp
public class OrderRepository : IOrderRepository
{
    private readonly AppDbContext _context;
    public OrderRepository(AppDbContext context) => _context = context;

    public Order Get(OrderId id) => _context.Orders
        .Include(o => o.Items)
        .FirstOrDefault(o => o.Id == id);

    public void Add(Order order) => _context.Orders.Add(order);
    public void Update(Order order) => _context.Orders.Update(order);
    public void Remove(Order order) => _context.Orders.Remove(order);
    public IEnumerable<Order> GetByCustomer(string customerName) =>
        _context.Orders.Where(o => o.CustomerName == customerName).ToList();
}
```

The domain layer uses the `IOrderRepository` interface, keeping it clean of infrastructure concerns.

---

## 27.6 Domain Events

A **Domain Event** is something that happened in the domain that domain experts care about. Events are used to capture side effects and to communicate between aggregates or bounded contexts.

### Characteristics

- Represented as immutable objects, named in the past tense (e.g., `OrderSubmitted`).
- Contain relevant data (e.g., `OrderId`, `SubmittedAt`).
- Are raised by aggregates and published to interested handlers.

### Example: OrderSubmitted Event

```csharp
public record OrderSubmitted
{
    public OrderId OrderId { get; }
    public DateTime SubmittedAt { get; }
    public string CustomerName { get; }

    public OrderSubmitted(OrderId orderId, string customerName)
    {
        OrderId = orderId;
        CustomerName = customerName;
        SubmittedAt = DateTime.UtcNow;
    }
}
```

### Raising Events in the Aggregate

We need a way to collect events inside the aggregate and then publish them. A common pattern is to have a list of events in the aggregate root.

```csharp
public abstract class AggregateRoot
{
    private readonly List<object> _domainEvents = new List<object>();
    public IReadOnlyList<object> DomainEvents => _domainEvents.AsReadOnly();

    protected void AddDomainEvent(object @event) => _domainEvents.Add(@event);
    public void ClearEvents() => _domainEvents.Clear();
}

public class Order : AggregateRoot
{
    // ... existing code

    public void Submit()
    {
        // ... validation
        Status = OrderStatus.Submitted;
        AddDomainEvent(new OrderSubmitted(Id, CustomerName));
    }
}
```

Then, after saving the aggregate, the repository or a dispatcher would publish the events to handlers (e.g., send email, update inventory).

Domain events promote loose coupling and are a great fit for eventual consistency scenarios.

---

## 27.7 Putting It All Together: An E‑Commerce Example

Let's build a small e‑commerce domain with orders, products, and a repository. We'll use DDD principles.

### Product (Value Object)

```csharp
public record Product
{
    public ProductId Id { get; }
    public string Name { get; }
    public Money Price { get; }

    public Product(ProductId id, string name, Money price)
    {
        Id = id;
        Name = name;
        Price = price;
    }
}

public record ProductId(Guid Value);
```

### OrderItem (Entity inside Order)

```csharp
public class OrderItem
{
    public Product Product { get; }
    public int Quantity { get; private set; }
    public Money Price { get; } // price at order time

    internal OrderItem(Product product, int quantity)
    {
        Product = product;
        Quantity = quantity;
        Price = product.Price; // copy current price
    }

    public void UpdateQuantity(int newQuantity)
    {
        if (newQuantity <= 0) throw new ArgumentException("Quantity must be positive");
        Quantity = newQuantity;
    }
}
```

### Order (Aggregate Root)

```csharp
public class Order : AggregateRoot
{
    public OrderId Id { get; }
    public string CustomerName { get; }
    public DateTime OrderDate { get; }
    private readonly List<OrderItem> _items = new List<OrderItem>();
    public IReadOnlyList<OrderItem> Items => _items.AsReadOnly();
    public OrderStatus Status { get; private set; }

    private Order() { } // for ORM

    public Order(string customerName)
    {
        Id = new OrderId(Guid.NewGuid());
        CustomerName = customerName;
        OrderDate = DateTime.UtcNow;
        Status = OrderStatus.Pending;
    }

    public void AddItem(Product product, int quantity)
    {
        if (Status != OrderStatus.Pending)
            throw new InvalidOperationException("Cannot modify submitted order");
        if (quantity <= 0) throw new ArgumentException("Quantity must be positive");

        var existing = _items.FirstOrDefault(i => i.Product.Id == product.Id);
        if (existing != null)
        {
            existing.UpdateQuantity(existing.Quantity + quantity);
        }
        else
        {
            _items.Add(new OrderItem(product, quantity));
        }
    }

    public void RemoveItem(ProductId productId)
    {
        if (Status != OrderStatus.Pending)
            throw new InvalidOperationException("Cannot modify submitted order");
        var item = _items.FirstOrDefault(i => i.Product.Id == productId);
        if (item != null) _items.Remove(item);
    }

    public void Submit()
    {
        if (Status != OrderStatus.Pending)
            throw new InvalidOperationException("Order already submitted");
        if (!_items.Any())
            throw new InvalidOperationException("Cannot submit empty order");

        Status = OrderStatus.Submitted;
        AddDomainEvent(new OrderSubmitted(Id, CustomerName));
    }
}
```

### Repository Interface

```csharp
public interface IOrderRepository
{
    Order Get(OrderId id);
    void Add(Order order);
    void Update(Order order);
    void Remove(OrderId id);
    IEnumerable<Order> GetByCustomer(string customerName);
}
```

### Domain Event

```csharp
public record OrderSubmitted
{
    public OrderId OrderId { get; }
    public string CustomerName { get; }
    public DateTime SubmittedAt { get; }

    public OrderSubmitted(OrderId orderId, string customerName)
    {
        OrderId = orderId;
        CustomerName = customerName;
        SubmittedAt = DateTime.UtcNow;
    }
}
```

### Using the Domain Model

```csharp
// In a service or application layer
public class OrderService
{
    private readonly IOrderRepository _orderRepository;
    private readonly IProductRepository _productRepository;
    private readonly IEventDispatcher _eventDispatcher;

    public OrderService(IOrderRepository orderRepository, IProductRepository productRepository, IEventDispatcher eventDispatcher)
    {
        _orderRepository = orderRepository;
        _productRepository = productRepository;
        _eventDispatcher = eventDispatcher;
    }

    public void CreateOrder(string customerName, List<(ProductId productId, int quantity)> items)
    {
        var order = new Order(customerName);
        foreach (var item in items)
        {
            var product = _productRepository.Get(item.productId);
            order.AddItem(product, item.quantity);
        }
        _orderRepository.Add(order);
        // Save changes (e.g., unit of work)
        // After saving, dispatch events
        foreach (var @event in order.DomainEvents)
        {
            _eventDispatcher.Dispatch(@event);
        }
        order.ClearEvents();
    }
}
```

---

## 27.8 Best Practices and Common Pitfalls

### 1. Keep the Domain Model Pure

The domain layer should have no dependencies on infrastructure (databases, frameworks, external services). Dependencies should point inward.

### 2. Use Factories for Complex Creation

If creating an aggregate requires complex logic or multiple steps, consider a factory (either a separate class or a static factory method on the aggregate).

### 3. Design Small Aggregates

Aggregates should be as small as possible while still maintaining consistency. Large aggregates can lead to performance issues and contention.

### 4. Avoid Injecting Repositories into Entities

Entities should not have references to repositories or other infrastructure. If they need to look up something, raise a domain event and let a handler deal with it.

### 5. Use Value Objects Extensively

Model concepts like money, date ranges, and addresses as value objects. This makes the domain richer and eliminates primitive obsession.

### 6. Domain Events for Cross‑Aggregate Communication

When a change in one aggregate needs to trigger something in another, use domain events and eventual consistency, not direct references.

### 7. Don't Over‑Apply DDD

DDD is best for complex domains. For simple CRUD applications, it may be overkill. Use it where the business logic is non‑trivial.

### 8. Collaborate with Domain Experts

DDD relies on deep understanding of the domain. Work closely with domain experts to refine the ubiquitous language and model.

---

## 27.9 Chapter Summary

In this chapter, you've learned the core tactical patterns of Domain‑Driven Design:

- **Entities** have identity and lifecycle.
- **Value Objects** are immutable and defined by their attributes.
- **Aggregates** group related objects under a root, enforcing consistency.
- **Repositories** provide a collection‑like interface for aggregates.
- **Domain Events** capture significant occurrences in the domain.

These patterns help you build models that are expressive, maintainable, and aligned with business needs. Combined with a ubiquitous language, DDD enables you to tackle complex domains effectively.

In the next chapter, **Securing Your Applications**, we'll shift focus to security – protecting your applications from common vulnerabilities, securing data, implementing authentication and authorization, and following best practices to keep your users and their data safe.

**Exercises:**

1. Identify a small domain in your experience (e.g., a library management system). Define the entities, value objects, and aggregates. Write the core classes in C#.
2. Extend the order example with a `Customer` aggregate and a domain event `OrderShipped`. Implement a handler that sends a notification.
3. Refactor a piece of code that uses primitives extensively (e.g., `decimal price`, `string currency`) to use value objects.
4. Discuss with a colleague or mentor about a real project where DDD patterns could have improved the design.

Now, get ready to secure your applications in Chapter 28!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='26. key_design_patterns_in_c.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='28. securing_your_applications.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
